# Mean Teacher Bert Assessment



# Requirement Installation 

In [2]:
#installing requirements
!pip install textattack==0.3.0

     |████████████████████████████████| 359 kB 8.5 MB/s 
     |████████████████████████████████| 776.8 MB 17 kB/s 
     |████████████████████████████████| 101 kB 12.0 MB/s 
     |████████████████████████████████| 59 kB 8.3 MB/s 
     |████████████████████████████████| 284 kB 46.9 MB/s 
     |████████████████████████████████| 20.1 MB 1.2 MB/s 
     |████████████████████████████████| 769 kB 53.1 MB/s 
     |████████████████████████████████| 264 kB 42.5 MB/s 
     |████████████████████████████████| 2.6 MB 57.7 MB/s 
     |████████████████████████████████| 895 kB 57.7 MB/s 
     |████████████████████████████████| 3.3 MB 53.3 MB/s 
     |████████████████████████████████| 636 kB 64.3 MB/s 
     |████████████████████████████████| 118 kB 76.7 MB/s 
     |████████████████████████████████| 243 kB 77.7 MB/s 
     |████████████████████████████████| 69 kB 8.8 MB/s 
     |████████████████████████████████| 64 kB 3.1 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel 

# Importing and Declaration

In [ ]:
import os 
os.chdir('/content/drive/MyDrive/Master Thesis/Mean-Teacher-BERT-assessment/')

In [ ]:
# importing libraries 
import os 
import pandas as pd 
import numpy as np
import tensorflow as tf

from ipywidgets import *
from IPython.display import display


from Utils.utils import label_to_int, dataset_split, convert_to_category, create_tokenizer, data_tokenization
from Utils.data_loader import data_loader
from Bert.bert_model import bert_model
from Attack_model.TextAttack import attack_model


from MeanTeacher.train_mean_teacher import train_mean_teacher

import random 
random.seed(42)

# Hyperparameter selections

In [ ]:
# Parameter declaration 
max_len=widgets.BoundedIntText(value=100,min=10,max=512,step=10,description='Max Length:',disabled=False,)

learning_rate= widgets.Dropdown(options=[2e-5, 1e-5, 1e-3],value=2e-5,description='Learning Rate:',disabled=False,)

epochs= widgets.Dropdown(options=[3, 4, 5,6],value=3,description='Epochs:',disabled=False,)

batch_size= widgets.Dropdown(options=[4, 5,6,7,8],value=4,description='Batch Size:',disabled=False,)

#for mean teacher model
alpha= widgets.BoundedFloatText(value=0.999,min=0.90,max=0.999,step=0.001,description='Alpha:',disabled=False, readout_format='.3f')
ratio= widgets.BoundedFloatText(value=0.5,min=0.1,max=0.9,step=0.1,description='Ratio:',disabled=False, readout_format='.2f')


widgets.HBox([max_len, learning_rate, epochs, batch_size,alpha,ratio])



In [ ]:
# Details related to the dataset
# columns names 
feature_col=widgets.Text(value='tweet',placeholder='Enter Feature Col Name',description='Feature Column:',disabled=False)
target_col=widgets.Text(value='label',placeholder='Enter Target Col Name',description='Target Column:',disabled=False) 
aug_col=widgets.Text(value='aug_tweet',placeholder='Enter Augmented data Col Name',description='Augment Column:',disabled=False)   


#pretrained model details
pretrained_weights=widgets.Dropdown(options=['bert-base-uncased','distilbert-base-uncased' ],value='bert-base-uncased',description='Pretrained Model:',disabled=False,) 


widgets.HBox([feature_col,target_col,aug_col,pretrained_weights])

In [ ]:
#Data location 
Data_loc=widgets.Text(value='Data/codalab/',placeholder='Enter Data location ',description=' Data Location Column:',disabled=False)   
widgets.HBox([Data_loc])

# Reading dataset

In [ ]:
df_train,df_test,df_aug_unlabel= data_loader(Data_loc.value,target_col.value,aug_col.value)

# Training 

This section is to train and save the model. <br>
__If model is already trained and saved, directly move forward for attack and evaluation section.__ 

## BERT Model Training

In [ ]:
tokenizer=create_tokenizer(pretrained_weights.value)
x_train,y_train,vocab_size= data_tokenization(df_train,feature_col.value,target_col.value,max_len.value,tokenizer)
x_test,y_test,_= data_tokenization(df_test,feature_col.value,target_col.value,max_len.value,tokenizer)

In [ ]:
bert=bert_model(pretrained_weights.value,max_len.value,learning_rate.value)
bert.summary()

In [ ]:
with tf.device('/GPU:0'):
    bert.fit(x_train,y_train,batch_size=batch_size.value,epochs=epochs.value,verbose=1)

In [ ]:
#saving the model 
bert.save_weights('SavedModels/BERT_model.h5')

## Adversarial Mean Teacher BERT Training

In [ ]:
# Training
tokenizer=create_tokenizer(pretrained_weights.value)
x_train,y_train,vocab_size= data_tokenization(df_train,feature_col.value,target_col.value,max_len.value,tokenizer)
x_test,y_test,_= data_tokenization(df_test,feature_col.value,target_col.value,max_len.value,tokenizer)
x_unlabel,_,_=data_tokenization(df_aug_unlabel,aug_col.value,target_col.value,max_len.value,tokenizer)

In [ ]:

with tf.device('/GPU:0'):
    student, teacher= train_mean_teacher(x_train, y_train, x_unlabel, 
                                         pretrained_weights.value,
                                         epochs.value,batch_size.value,
                                         learning_rate.value,max_len.value,alpha.value)

In [ ]:
teacher.save_weights('SavedModels/BERT_teacher.h5')

# Attack And Evaluation

In [ ]:
import textattack

In [ ]:
bert_to_attack=widgets.Textarea(value='SavedModels/Codalab/DistillBERT_model.h5',placeholder='Enter Model location ',description=' BERT Model location:',disabled=False)
MTbert_to_attack=widgets.Textarea(value='SavedModels/Codalab/DistillBERT_teacher.h5',placeholder='Enter Model location ',description=' MTBERT Model location:',disabled=False)   
pretrained_weights=widgets.Dropdown(options=['bert-base-uncased','distilbert-base-uncased' ],value='bert-base-uncased',description='Pretrained Model:',disabled=False,) 
model_name=widgets.Dropdown(options=['BERT','DistillBERT']
                               ,value='BERT',
                               description='Model Name:',
                               disabled=False,) 

tf.print('\n Please select the location of BERT and MT BERT model.\n')
display(bert_to_attack, MTbert_to_attack)
tf.print('\n Please select the pretrained model weights.')
display(pretrained_weights)
tf.print('\n model_name: Model name for attack.Note : Please confirm with the bert_to_attack parameter')
display(model_name)


 Please select the location of BERT and MT BERT model.



Textarea(value='SavedModels/Codalab/DistillBERT_model.h5', description=' BERT Model location:', placeholder='E…

Textarea(value='SavedModels/Codalab/DistillBERT_teacher.h5', description=' MTBERT Model location:', placeholde…


 Please select the pretrained model weights.


Dropdown(description='Pretrained Model:', options=('bert-base-uncased', 'distilbert-base-uncased'), value='ber…


 model_name: Model name for attack.Note : Please confirm with the bert_to_attack parameter


Dropdown(description='Model Name:', options=('BERT', 'DistillBERT'), value='BERT')

In [ ]:
# attack parameters 

# Query Budget is maximum query allowed
query_budget= widgets.BoundedIntText(value=200,min=10,max=500,step=20,description='Query Budget:',disabled=False,)

# number of example to take for attack from test dataset randomly 
num_examples=widgets.BoundedIntText(value=200,min=10,max=len(df_test),step=20,description='Number of Example:',disabled=False,)

# attack recipes 
attack_recipe=widgets.Dropdown(options=['TextFoolerJin2019','PWWSRen2019','BERTAttackLi2020','BAEGarg2019']
                               ,value='TextFoolerJin2019',
                               description='Attack Recipe:',
                               disabled=False,) 


tf.print('\n query_budget: Maximum Query allowed to attack model')
display(query_budget)
tf.print('\n num_examples: Number of example to take for attack from test dataset randomly. ')
display(num_examples)
tf.print('\n attack_recipe: Attack methodolgy to attack the model. ')
display(attack_recipe)



 query_budget: Maximum Query allowed to attack model


BoundedIntText(value=200, description='Query Budget:', max=500, min=10, step=20)


 num_examples: Number of example to take for attack from test dataset randomly. 


BoundedIntText(value=200, description='Number of Example:', max=2040, min=10, step=20)


 attack_recipe: Attack methodolgy to attack the model. 


Dropdown(description='Attack Recipe:', options=('TextFoolerJin2019', 'PWWSRen2019', 'BERTAttackLi2020', 'BERTA…

- __PWWSRen 2019 - Generating Natural Language Adversarial Examples through Probability Weighted Word Saliency. Shuhuai Ren, Yihe Deng, Kun He, Wanxiang Che__<br>
Based on the synonyms substitution strategy, they introduced a new word replacement order determined by both the word saliency and the classification probability, and propose a greedy algorithm called probability weighted word saliency (PWWS) for text adversarial attack.
- __TextFoolerJin2019 - Is BERT Really Robust? A Strong Baseline for Natural Language Attack on Text Classification and Entailment.__<br>
In this paper, we present TextFooler, a simple but strong baseline to generate natural adversarial text. By applying it to two fundamental natural language tasks, text classification and textual entailment, we successfully attacked three target models, including the powerful pre-trained BERT, and the widely used convolutional and recurrent neural networks. We demonstrate the advantages of this framework in three ways: (1) effective---it outperforms state-of-the-art attacks in terms of success rate and perturbation rate, (2) utility-preserving---it preserves semantic content and grammaticality, and remains correctly classified by humans, and (3) efficient---it generates adversarial text with computational complexity linear to the text length. *The code, pre-trained target models, and test examples are available at https://github.com/jind11/TextFooler.
- __BAEGarg2019-BAE (BAE: BERT-Based Adversarial Examples)__ <br>
BAE, a black box attack for generating adversarial examples using contextual perturbations from a BERT masked language model. BAE replaces and inserts tokens in the original text by masking a portion of the text and leveraging the BERT-MLM to generate alternatives for the masked tokens. Through automatic and human evaluations, we show that BAE performs a stronger attack, in addition to generating adversarial examples with improved grammaticality and semantic coherence as compared to prior work.
- __BERTAttackLi2020 - BERT-ATTACK: Adversarial Attack Against BERT Using BERT.__ <br>
we propose \textbf{BERT-Attack}, a high-quality and effective method to generate adversarial samples using pre-trained masked language models exemplified by BERT. We turn BERT against its fine-tuned models and other deep neural models for downstream tasks. Our method successfully misleads the target models to predict incorrectly, outperforming state-of-the-art attack strategies in both success rate and perturb percentage, while the generated adversarial samples are fluent and semantically preserved. Also, the cost of calculation is low, thus possible for large-scale generations.


In [ ]:
'''
Some of the attack recipe provided by textattack is not working with lengthy article. So, 
I am filtering the with the length. May be in future I will remove it.
'''
df_test['len']=df_test.apply(lambda row: len(row[feature_col.value].split()),axis=1)
df_test_for_attack= df_test[(df_test['len']>0) & (df_test['len']<100)][[feature_col.value,target_col.value]].dropna().reset_index(drop=True)
df_test_for_attack=df_test_for_attack.sample(frac=1).reset_index(drop=True)
print('Length of dataset', len(df_test_for_attack))


#creating dataset for the attacking
dataset_for_attack=list(df_test_for_attack.itertuples(index=False, name=None))
dataset_for_attack = textattack.datasets.Dataset(dataset_for_attack)


Length of dataset 2040


### BERT model

In [ ]:
bert= bert_model(pretrained_weights.value,max_len.value,learning_rate.value) 
bert.load_weights(bert_to_attack.value)

attack_model(bert, 
             dataset_for_attack,
             pretrained_weights.value,
             max_len.value,
             model_name.value,
             attack_recipe=attack_recipe.value,
             query_budget= query_budget.value,
             num_examples=num_examples.value )

Some layers from the model checkpoint at bert-base-uncased were not used when initializing TFBertModel: ['nsp___cls', 'mlm___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at bert-base-uncased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


The parameters `output_attentions`, `output_hidden_states` and `use_cache` cannot be updated when calling a model.They have to be set to True/False in the config object (i.e.: `config=XConfig.from_pretrained('name', output_attentions=True)`).


The parameter `return_dict` cannot be set in graph mode and will always be set to `True`.
textattack: Unknown if model of class <class 'tensorflow.python.keras.engine.functional.Functional'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path Result/TextFoolerJin2019/BERT_nexp1500_qb200_2021-07-31_04:40.csv
  0%|          | 0/1500 [00:00<?, ?it/s]

Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.5
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.840845057
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']
       

[Succeeded / Failed / Skipped / Total] 0 / 1 / 0 / 1:   0%|          | 1/1500 [00:05<2:21:46,  5.67s/it]

...

--------------------------------------------- Result 1498 ---------------------------------------------
1 (73%) --> 0 (62%)

a quote by doctor alberto zangrillo saying that covid is a consequence of immigration

a commendation by doctor hugh zangrillo commenting that covid is a consequence of immigration




[Succeeded / Failed / Skipped / Total] 972 / 448 / 79 / 1499: 100%|█████████▉| 1499/1500 [36:39<00:01,  1.47s/it]

--------------------------------------------- Result 1499 ---------------------------------------------
1 (73%) --> [FAILED]

world first covid vaccine in russia given to putins daughter russianvaccine




[Succeeded / Failed / Skipped / Total] 973 / 448 / 79 / 1500: 100%|██████████| 1500/1500 [36:39<00:00,  1.47s/it]

--------------------------------------------- Result 1500 ---------------------------------------------
0 (57%) --> 1 (71%)

when we know the actual number is much higher a we see here on a sonoma county dashboard

when we know the actual nombre is much higher a we see here on a sonoma county dashboard



+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 973    |
| Number of failed attacks:     | 448    |
| Number of skipped attacks:    | 79     |
| Original accuracy:            | 94.73% |
| Accuracy under attack:        | 29.87% |
| Attack success rate:          | 68.47% |
| Average perturbed word %:     | 18.76% |
| Average num. words per input: | 25.37  |
| Avg num queries:              | 138.52 |
+-------------------------------+--------+


### MTBERT model

In [ ]:
bert_teacher= bert_model(pretrained_weights.value,max_len.value,learning_rate.value) 
bert_teacher.load_weights(MTbert_to_attack.value)

attack_model(bert_teacher, 
             dataset_for_attack,
             pretrained_weights.value,
             max_len.value,
             'MT'+model_name.value,
             attack_recipe=attack_recipe.value,
             query_budget= query_budget.value,
             num_examples=num_examples.value )

Some layers from the model checkpoint at bert-base-uncased were not used when initializing TFBertModel: ['nsp___cls', 'mlm___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at bert-base-uncased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


The parameters `output_attentions`, `output_hidden_states` and `use_cache` cannot be updated when calling a model.They have to be set to True/False in the config object (i.e.: `config=XConfig.from_pretrained('name', output_attentions=True)`).


The parameter `return_dict` cannot be set in graph mode and will always be set to `True`.
textattack: Unknown if model of class <class 'tensorflow.python.keras.engine.functional.Functional'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path Result/TextFoolerJin2019/MTBERT_nexp1500_qb200_2021-07-31_05:16.csv
  0%|          | 0/1500 [00:00<?, ?it/s]

Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.5
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.840845057
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']
       

[Succeeded / Failed / Skipped / Total] 0 / 1 / 0 / 1:   0%|          | 1/1500 [00:06<2:30:07,  6.01s/it]

...

--------------------------------------------- Result 1498 ---------------------------------------------
1 (64%) --> [FAILED]

a quote by doctor alberto zangrillo saying that covid is a consequence of immigration




[Succeeded / Failed / Skipped / Total] 540 / 888 / 71 / 1499: 100%|█████████▉| 1499/1500 [41:54<00:01,  1.68s/it]

--------------------------------------------- Result 1499 ---------------------------------------------
1 (63%) --> [FAILED]

world first covid vaccine in russia given to putins daughter russianvaccine




[Succeeded / Failed / Skipped / Total] 541 / 888 / 71 / 1500: 100%|██████████| 1500/1500 [41:55<00:00,  1.68s/it]

--------------------------------------------- Result 1500 ---------------------------------------------
0 (65%) --> 1 (56%)

when we know the actual number is much higher a we see here on a sonoma county dashboard

when we know the actual nombre is much higher a we see here on a sonoma county dashboard



+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 541    |
| Number of failed attacks:     | 888    |
| Number of skipped attacks:    | 71     |
| Original accuracy:            | 95.27% |
| Accuracy under attack:        | 59.2%  |
| Attack success rate:          | 37.86% |
| Average perturbed word %:     | 18.01% |
| Average num. words per input: | 25.37  |
| Avg num queries:              | 159.84 |
+-------------------------------+--------+
